# Search Agent

In [ ]:

import os
import sys

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

import requests
import json
from dotenv import load_dotenv

from langchain.prompts import PromptTemplate, load_prompt
from langchain.agents import load_tools ,initialize_agent, AgentExecutor, create_self_ask_with_search_agent
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool




from utils.sambanova_endpoint import SambaNovaEndpoint, SambaverseEndpoint

load_dotenv("../../.env")

from langchain.globals import set_verbose

#set_verbose(True)

from langchain.globals import set_debug

set_debug(True)

##  Define LLM

### sambanova

In [ ]:
#sambeverse llm
llm = SambaverseEndpoint(
            sambaverse_model_name="Mistral/Mistral-7B-Instruct-v0.2",
            model_kwargs={
                "do_sample": False, 
                "max_tokens_to_generate": 500,
                "temperature": 0.01,
                "top_p": 1,
                "process_prompt": False,
                "select_expert": "Mistral-7B-Instruct-v0.2"
            }
        )

#sambastudio llm
#llm = SambaNovaEndpoint(
#    model_kwargs={"do_sample": False, "temperature": 0.0},
#)

## Tools

In [ ]:
#Base tools searcha nd calculator
tool_names =['llm-math']#,'serpapi',]
tools=load_tools(tool_names,llm)
tools

In [ ]:
# new tool basic llm
class BasicLLMInput(BaseModel):
    query: str = Field(description="raw user interaction")


def queryLLM(query: str) -> str:
    """Process a query with an LLM"""
    prompt = load_prompt(os.path.join(kit_dir,"prompts/llama70b-Q&A.yaml"))
    query = prompt.format(question=query)
    return llm.invoke(query)

askLLM = StructuredTool.from_function(
    func=queryLLM,
    name="conversational_query",
    description="process user input conversation, following conversation without factual checking",
    args_schema=BasicLLMInput,
    return_direct=True,
)

tools.append(askLLM)

In [ ]:
#define serper tool
class SerperInput(BaseModel):
    query: str = Field(description="google search query")


def querySerper(query: str) -> str:
    """A search engine. Useful for when you need to answer questions about current events. Input should be a search query."""
    url = "https://google.serper.dev/search"
    payload = json.dumps({
        "q": query
    })
    headers = {
        'X-API-KEY': os.environ.get("SERPER_API_KEY"),
        'Content-Type': 'application/json'
    }

    response = requests.post(url, headers=headers, data=payload)
    prompt = load_prompt(os.path.join(kit_dir, "prompts/llama70b-SearchAnalysis.yaml"))
    formated_prompt = prompt.format(question=query, context=json.dumps(response.json()))
    
    return(llm.invoke(formated_prompt))
   
serper = StructuredTool.from_function(
    func=querySerper,
    name="search_engine",
    description="A search engine. Useful for when you need to answer questions about current events. Input should be a search query.",
    args_schema=SerperInput,
    return_direct=False,
) 

tools.append(serper)


In [ ]:
#OpenSERP https://github.com/karust/openserp?tab=readme-ov-file
#Run docker run -p 127.0.0.1:7000:7000 -it karust/openserp serve -a 0.0.0.0 -p 7000

class OpenSerpInput(BaseModel):
    query: str = Field(description="google search query")


def queryOpenSerp(query: str) -> str:
    """A search engine. Useful for when you need to answer questions about current events. Input should be a search query."""
    url = "http://127.0.0.1:7000/google/search"
    params = {
        "lang": "EN",
        "limit": 10,
        "text": query
    }

    response = requests.get(url, params=params)
 
    prompt = load_prompt(os.path.join(kit_dir, "prompts/llama70b-OpenSearchAnalysis.yaml"))
    formated_prompt = prompt.format(question=query, context=json.dumps(response.json()))
    
    return(llm.invoke(formated_prompt))
   
openSerp = StructuredTool.from_function(
    func=queryOpenSerp,
    name="search_engine",
    description="A search engine. Useful for when you need to answer questions about current events. Input should be a search query.",
    args_schema=OpenSerpInput,
    return_direct=False,
) 

tools.append(openSerp)


In [ ]:
tools

## prompting experiments

### Base prompt

In [ ]:

PREFIX = """<s>[INST] Answer the following questions as best you can. You have access to the following tools:"""

FORMAT_INSTRUCTIONS = """Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of {tool_names}
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question"""

SUFFIX = """Begin!

Question: {input} 
Thought:{agent_scratchpad} [/INST]"""

### modified prompt

In [ ]:

PREFIX = "[INST] <<SYS>> You are smart assistant that selects a function from list of functions based on user questions.\
Run only one Action at a time. \nAnswer the following questions as best you can. You have access to the following tools:"

FORMAT_INSTRUCTIONS = "Use the following format:\n\n\
Question: the input question you must answer\n\
Thought: you should always think about what to do\n\
Action: the action to take, should be one of [Search, Calculator]\n\
Action Input: the input to the action\n\
Observation: the result of the action\n\
... (this Thought/Action/Action Input/Observation can repeat N times, but only one at a time)\n\
Thought: I now know the final answer\n\
Final Answer: the final answer to the original input question"

SUFFIX = """Stop after each Action call and wait to get the intermediate results, do not make up multiple steps by yourself

Begin! <</SYS>>

Question: {input} 
Thought:{agent_scratchpad}[/INST]"""

### Improved prompt

In [ ]:
PREFIX = """[INST] <<SYS>> You are smart assistant that selects a function from list of functions based on user questions.Run only one Action at a time.
You have access to the following tools:"""

FORMAT_INSTRUCTIONS = """Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do (only one at a time)
Action: the action to take, should be one of [Search, Calculator] 
Action Input: the input to the action

then you will get an observation
if your Thought is:  I now know the final answer
you should reply
Final Answer: the final answer to the original input question"""

SUFFIX = """Stop after each Action call and wait to get the intermediate results, do not make up multiple steps by yourself

example:
Question:  How much money does the richest person in the world have?
Though: I'll need to find the current richest person in the world, then i need to find the net worth of this person
Action: Search
Action input: richest person in the world

Begin! <</SYS>>

Question: {input} 
Thought:{agent_scratchpad}[/INST]"""

In [ ]:
"""
<s>[INST] Answer the following questions as best you can. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of Search, Calculator
Action Input: the input to the action
Observation: the result of the action (wait for an observation, you must not provide an observation)
... (this Thought/Action/Action Input/Observation can repeat N times Only if an observation was provided)
Thought: I now know the final answer 
Final Answer: the final answer to the original input question Only if there is enough information to give a response

Begin!

Question: who is older? the president of America or PM of India. give me their difference in age 
Thought: [/INST]
"""

In [ ]:
"""<s>[INST] You are a helpfull assistant who can use tools one at a time to find an answer. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of Search, Calculator
Action Input: the input to the action and then </s>
Observation: the result of the action (wait for an observation, you must not provide an observation)
... (this Thought/Action/Action Input/Observation can repeat N times Only if an observation was provided)
Final Answer: the final answer to the original input question, Only if there is enough information to give a response

Don't try to make up observations  stop if an observation is needed

Begin!

Question: who is older? the president of America or PM of India. give me their difference in age 
Thought: [/INST]
"""

In [ ]:
"""<s>[INST] You are a helpful assistant who can use tools one at a time to  get closer to the answer,. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of Search, Calculator
Action Input: the input to the action
Observation: the result of the action (wait for an observation, you must not provide an observation)
... (this Thought/Action/Action Input/Observation can repeat N times Only if an observation was provided)

if there is not an answer based in observations yet reply (waiting for an observation) and finish your response
if there is enough information to give a response reply
Final Answer: the final answer to the original input question 
if not write (...)

Don't try to make up observations  stop if an observation is needed
Think step by step and be patient when answering.
Only when you are done with all steps, provide the answer based on the intermediate steps.  
Before presenting your final answer, make sure that you correctly waited for the observation steps. 

Begin!

Question: who is older? the president of America or PM of India. give me their difference in age 
Thought: [/INST]
"""

In [ ]:
PREFIX = """<s>[INST] You are a helpful assistant who can use tools one at a time to  get closer to the answer,. You have access to the following tools:"""

FORMAT_INSTRUCTIONS = """Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of {tool_names}
Action Input: the input to the action
Observation: the result of the action (wait for an observation, you must not provide an observation)
... (this Thought/Action/Action Input/Observation can repeat N times Only if an observation was provided)

if there is not an answer based in observations yet reply (waiting for an observation) and finish your response
if there is enough information to give a response reply
Final Answer: the final answer to the original input question 
if not write (...)
"""

SUFFIX = """Don't try to make up observations  stop if an observation is needed
Think step by step and be patient when answering.
Only when you are done with all steps, provide the answer based on the intermediate steps.  
Before presenting your final answer, make sure that you correctly waited for the observation steps. 

Begin!

Begin! <</SYS>>

Question: {input} 
Thought:{agent_scratchpad}[/INST]"""

## Agent 

In [ ]:
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    #verbose=True,
    max_iterations = 4,
    agent_kwargs={
        #'prefix': PREFIX, 
        #'format_instructions': FORMAT_INSTRUCTIONS,
        #'suffix': SUFFIX
    }
)

In [ ]:
agent.invoke("hi how are you",)

In [ ]:
agent.invoke("who is older? the president of America or PM of India. give me their difference in age",)


In [ ]:
agent.invoke("who is the president of Colombia")


# sambanova mistral response 

In [ ]:
agent.invoke("who is older? the president of America or PM of India. give me their difference in age",)
